In [12]:
import numpy as np

In [13]:
Q = [4, 1, 2, 3, 3, 5, 8]
C = [4, 1, 1, 2, 3, 5, 7, 9]

In [14]:
# 原始的DTW算法
# 时间复杂度O(m*n) len(s1):m, len(s2):n
def DTWDistance(s1, s2):
    DTW={}

    for i in range(len(s1)):
        DTW[(i, -1)] = float('inf')
    for i in range(len(s2)):
        DTW[(-1, i)] = float('inf')
    DTW[(-1, -1)] = 0

    for i in range(len(s1)):
        for j in range(len(s2)):
            dist= (s1[i]-s2[j])**2
            DTW[(i, j)] = dist + min(DTW[(i-1, j)],DTW[(i, j-1)], DTW[(i-1, j-1)])

#     print(DTW)
    return np.sqrt(DTW[len(s1)-1, len(s2)-1])

In [15]:
DTWDistance(Q,C)

1.4142135623730951

In [16]:
# 进行优化, 采用窗口约束, sakoe-chiba band. 降低计算量. 
# 时间复杂度, O(w*s1), 
def DTWDistance_1(s1, s2, w):
    DTW={}

    w = max(w, abs(len(s1)-len(s2)))

    for i in range(-1,len(s1)): 
        for j in range(-1,len(s2)):
            DTW[(i, j)] = float('inf')
    DTW[(-1, -1)] = 0

    for i in range(len(s1)):
        for j in range(max(0, i-w), min(len(s2), i+w)):
            dist= (s1[i]-s2[j])**2
            DTW[(i, j)] = dist + min(DTW[(i-1, j)],DTW[(i, j-1)], DTW[(i-1, j-1)])
#     print(DTW)
    return np.sqrt(DTW[len(s1)-1, len(s2)-1])

In [17]:
DTWDistance_1(Q,C, 3)

1.4142135623730951

In [18]:
# LB_Keogh: 优先计算s1的上下界, U, L
#           U = max[q(i-r):q(i+2)], L = min[q(i-r):q(i+2)]
#           再计算s2超出s1上下界的值的时间部的距离累积和
# 时间复杂度O(n)
def LB_Keogh(s1,s2,r):
    LB_sum=0
    for ind,i in enumerate(s1):

        lower_bound=min(s2[(ind-r if ind-r>=0 else 0):(ind+r)])
        upper_bound=max(s2[(ind-r if ind-r>=0 else 0):(ind+r)])

        if i>upper_bound:
            LB_sum=LB_sum+(i-upper_bound)**2
        elif i<lower_bound:
            LB_sum=LB_sum+(i-lower_bound)**2

    return np.sqrt(LB_sum)

In [32]:
LB_Keogh(Q,C,2)

0.0

In [24]:
# LB_Kim: 求起始点，终止点，最大点，最小点之间距离的最大值, 时间复杂度O(n)
# LB_Kim_FL: 只计算首尾两点, 时间复杂度O(1)
def LB_Kim(s1,s2):
    first = (s1[0] - s2[0])**2
    last = (s1[-1] - s2[-1])**2
    max_dist = (max(s1)-max(s2))**2
    min_dist = (min(s1)-min(s2))**2
    return np.sqrt(max(first, last, max_dist, min_dist))
def LB_Kim_FL(s1,s2):
    first = (s1[0] - s2[0])**2
    last = (s1[-1] - s2[-1])**2
    return np.sqrt(max(first, last))

In [25]:
LB_Kim(Q,C)

1.0

In [30]:
LB_Kim_FL(Q,C)

1.0